# Primitives examples

The examples in this section illustrate some common ways to use primitives. Before running these examples, follow the instructions in [Install and set up.](install-qiskit)

<Admonition type="note">
    These examples all use the primitives from Qiskit Runtime, but you could use the base primitives instead.
</Admonition>

## Estimator examples

Efficiently calculate and interpret expectation values of the quantum operators required for many algorithms with Estimator. Explore uses in molecular modeling, machine learning, and complex optimization problems.

### Run a single experiment

Use Estimator to determine the expectation value of a single circuit-observable pair.

In [1]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
observable = SparsePauliOp("Z" * 50)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

estimator = Estimator(mode=backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: -0.004464285714285714
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


### Run multiple experiments in a single job

Use Estimator to determine the expectation values of multiple circuit-observable pairs.

In [2]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]

pubs = []
circuits = [IQP(mat) for mat in mats]
observables = [
    SparsePauliOp("X" * 50),
    SparsePauliOp("Y" * 50),
    SparsePauliOp("Z" * 50),
]

# Get ISA circuits
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

for qc, obs in zip(circuits, observables):
    isa_circuit = pm.run(qc)
    isa_obs = obs.apply_layout(isa_circuit.layout)
    pubs.append((isa_circuit, isa_obs))

estimator = Estimator(backend)
job = estimator.run(pubs)
job_result = job.result()

for idx in range(len(pubs)):
    pub_result = job_result[idx]
    print(f">>> Expectation values for PUB {idx}: {pub_result.data.evs}")
    print(f">>> Standard errors for PUB {idx}: {pub_result.data.stds}")

>>> Expectation values for PUB 0: -0.6333333333333333
>>> Standard errors for PUB 0: 0.7058575315471395
>>> Expectation values for PUB 1: -0.3
>>> Standard errors for PUB 1: 1.7169576449893864
>>> Expectation values for PUB 2: 0.0087890625
>>> Standard errors for PUB 2: 0.016871336080016838


### Run parameterized circuits

Use Estimator to run three experiments in a single job, leveraging parameter values to increase circuit reusability.

In [3]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Step 1: Map classical inputs to a quantum problem
theta = Parameter("θ")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)

number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
individual_phases = [[ph] for ph in phases]

ZZ = SparsePauliOp.from_list([("ZZ", 1)])
ZX = SparsePauliOp.from_list([("ZX", 1)])
XZ = SparsePauliOp.from_list([("XZ", 1)])
XX = SparsePauliOp.from_list([("XX", 1)])
ops = [ZZ, ZX, XZ, XX]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
chsh_isa_circuit = pm.run(chsh_circuit)
isa_observables = [
    operator.apply_layout(chsh_isa_circuit.layout) for operator in ops
]

# Step 3: Execute using Qiskit primitives.

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((4, 1))

estimator = Estimator(backend, options={"default_shots": int(1e4)})
job = estimator.run([(chsh_isa_circuit, reshaped_ops, individual_phases)])
# Get results for the first (and only) PUB
pub_result = job.result()[0]
print(f">>> Expectation values: {pub_result.data.evs}")
print(f">>> Standard errors: {pub_result.data.stds}")
print(f">>> Metadata: {pub_result.metadata}")

>>> Expectation values: [[ 1.25754065  1.18600782  0.9952536   0.73263992  0.37072952 -0.03331666
  -0.41809144 -0.758444   -1.01289183 -1.21540487 -1.2676663  -1.18404802
  -0.98284151 -0.69671018 -0.33969929  0.036583    0.4138452   0.78816769
   1.0328165   1.1935204   1.25100797]
 [ 0.01045229  0.40306628  0.76171034  1.03902255  1.20364605  1.23859588
   1.1781686   1.00113301  0.71434842  0.38869438 -0.036583   -0.41449846
  -0.79306719 -1.02693709 -1.22455062 -1.24970144 -1.18502792 -0.99558023
  -0.68952424 -0.37072952  0.03886944]
 [-0.02090457 -0.41319193 -0.72218763 -0.99002746 -1.16249018 -1.210832
  -1.1732691  -0.96716308 -0.69899662 -0.3436189  -0.00653268  0.38738785
   0.71990119  0.98316815  1.16608315  1.2320632   1.15105799  0.96552991
   0.67155937  0.33283998 -0.03821617]
 [ 1.2310833   1.15955047  0.95213792  0.67972522  0.35276465 -0.02515081
  -0.40437281 -0.74047913 -0.99296716 -1.16412335 -1.210832   -1.14942482
  -0.96291684 -0.68397146 -0.34067919  0.055201

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

In [4]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    EstimatorV2 as Estimator,
)

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = IQP(mat)
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = IQP(mat)
observable = SparsePauliOp("X" * 50)
another_observable = SparsePauliOp("Y" * 50)

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
another_isa_observable = another_observable.apply_layout(
    another_isa_circuit.layout
)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    estimator.options.resilience_level = 1

    job = estimator.run([(isa_circuit, isa_observable)])
    another_job = estimator.run(
        [(another_isa_circuit, another_isa_observable)]
    )
    result = job.result()
    another_result = another_job.result()

    # first job
    print(f" > Expectation value: {result[0].data.evs}")
    print(f" > Metadata: {result[0].metadata}")

    # second job
    print(f" > Another Expectation value: {another_result[0].data.evs}")
    print(f" > More Metadata: {another_result[0].metadata}")

 > Expectation value: 0.022123893805309734
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}
 > Another Expectation value: -0.2032258064516129
 > More Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


<span id="sampler-examples"></span>
## Sampler examples

Generate entire error-mitigated quasi-probability distributions sampled from quantum circuit outputs. Leverage Sampler’s capabilities for search and classification algorithms like Grover’s and QVSM.

### Run a single experiment

Use Sampler to return the measurement outcome as bitstrings or counts of a single circuit.

In [5]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]

print(f" > First ten results: {pub_result.data.meas.get_bitstrings()[:10]}")

 > First ten results: ['0101010011101011010010000011110001110100001100111011010110110000110000101100001010110001001001110000100101100000110101001001100', '1001101001101111001001100011011100111011010001011111101110101100000000111010010111000001011110110001110000100010101001100011010', '1000110111001010111101100100000011110100010001111001010011011001100000111011011100000101000100010100000000011000110100111100001', '0100011000010100100100001111000110100010000001001110010100001100100000001100000110111000001000111000001010100011110101101110000', '0011101001000001100011100011101011011000000000000100000011100101110100010001000111110001001001010011010010100110100100101100100', '1011011011011101100110011100100011100000001101001010000011011011111100111111011011010110111001001011100011111110001100100010010', '1111111011100111101110011000110110111010010011010110001000001011001011110000010101000000101101010101001001000010111000111011100', '00111000101100011101010011101010001110000010011110000110011

### Run multiple experiments in a single job

Use Sampler to return the measurement outcome as bitstrings or counts of multiple circuits in one job.

In [6]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]
circuits = [IQP(mat) for mat in mats]
for circuit in circuits:
    circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuits = pm.run(circuits)

sampler = Sampler(mode=backend)
job = sampler.run(isa_circuits)
result = job.result()

for idx, pub_result in enumerate(result):
    print(
        f" > First ten results for pub {idx}: {pub_result.data.meas.get_bitstrings()[:10]}"
    )

 > First ten results for pub 0: ['0111011100001101010010111110001101010011100011101011111111000100010100001011001100000000111011000010001010101110101100100000000', '1000101011011100010110000010110110010011011000000100110001111100010101010000000110100001010001010010010000110010010010011011010', '0001001001001100010111111100000111101010110101011010001101010101000110110001000110000000010101011101000001010100010101101011010', '0011010111110110010000000000010000101000001001101010100001110110010000011111111101000100010101010100000000111010100110101100001', '0010000111110001111000001100110101000011101000100000101000100110101100000001100010001011000101000001000010110000000010010000100', '0000001011100001000000110111100101000111110010001101000111111010011001110111000010110001010011010001001010000010001101011010000', '1010010010110101010100001110010000010111100010001010101001010011000100001101010100001111111100010110011010100000000011010000100', '0000000110001101010010111111100100110011111101001

### Run parameterized circuits

Run several experiments in a single job, leveraging parameter values to increase circuit reusability.

In [7]:
import numpy as np
from qiskit.circuit.library import RealAmplitudes
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

# Step 1: Map classical inputs to a quantum problem
circuit = RealAmplitudes(num_qubits=n_qubits, reps=2)
circuit.measure_all()

# Define three sets of parameters for the circuit
rng = np.random.default_rng(1234)
parameter_values = [
    rng.uniform(-np.pi, np.pi, size=circuit.num_parameters) for _ in range(3)
]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

# Step 3: Execute using Qiskit primitives.
sampler = Sampler(backend)
job = sampler.run([(isa_circuit, parameter_values)])
result = job.result()
# Get results for the first (and only) PUB
pub_result = result[0]
# Get counts from the classical register "meas".
print(
    f" >> First ten results for the meas output register: {pub_result.data.meas.get_bitstrings()[:10]}"
)

 >> First ten results for the meas output register: ['0000010000101001101010010100011100110100010111111110001001001011010010100110111000011101000000101111110001111011111011011000111', '0100011000010011001110001100010011110001111000000100000110100101100000001100101010100000010001000101110101010101001011010000101', '1100111000010011000010000100001111100100110010110000111100011000010010000100001100001110000011000100000001001000010001110110101', '0001101100011010111000100100110111110000110110011110111011101101111100011010010000011100000011100000101001101000000111100110101', '1100010100001101001010010100001100110011100111100011110110010011110101000010111000100000111001101001001111011111111010101000011', '0001100000110111110111100100110110110100001000111001001000000011110011001011100101101110010001111010010001111000100001111000110', '0111001010011111100000010000100111100100001000010011001000011001010100101101110010010000101011011001010001101111000110001011111', '11010000011011111000100111010

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

In [8]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.quantum_info import random_hermitian
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = IQP(mat)
circuit.measure_all()
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = IQP(mat)
another_circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run([isa_circuit])
    another_job = sampler.run([another_isa_circuit])
    result = job.result()
    another_result = another_job.result()

# first job

print(
    f" > The first ten measurement results of job 1: {result[0].data.meas.get_bitstrings()[:10]}"
)

 > The first ten measurement results of job 1: ['0101011110110000111010110010100000100011111011001101101100100010001010011010000010100010010101010100101010000010010000001001101', '0110011011111110000110100100001110101111001010101100100010000010010001011110000001010000010000001001111001000010010000011000110', '1011110000101101110001011000101001000101100001000111011010010010010110110111000010010000000000100001010110100000100010001000101', '1001010111000010001011101101000101110000000101001010111010110111100000101010010001010001100111110000000010000000000001001101010', '0100110010101000101000010011111000010011101110011111111010010010010010001110000100110001001000010000011110100100000101001001111', '1101101010100010001100000001000000101001000001111100111101111111001000101000001010111010010000010000111011000011100110001001011', '0100011100010010011011100010010010001000111001011111011100011000011111010001000000100001101101011101101111001000010101110001100', '0101011001001001100110101101101110

In [9]:
# second job
print(
    " > The first ten measurement results of job 2:",
    another_result[0].data.meas.get_bitstrings()[:10],
)

 > The first ten measurement results of job 2: ['0001110011000000000001100111101100111001111011010001010111110001000011010101001110011101111110011100101100000010100000110001100', '1100001010100100100001001001100000100001000000111001000010100010100000101010100010101100010111000110010101010101001010000011001', '0100111011110010101101111001010000011101010001010000011011000010100001101010001100110000010000001001001010010111100001010010010', '1110000010110110101101111000001101000000110010010000111100110100000100001001001000010100110100100110110101101101000000001001101', '0110011010110111000010010011000001101100010111010010110101100101000100000111100101100010010001000110001011000100001010000101100', '0001010111101101100010101110101111001000010101011000101001101001100101010111000100001000100001001001000110101001101010111000111', '0001101011010101100111111000100011000100010000011000010010000100100101110101011110010100101111100001101101000010101001011011100', '0100100000100010011001001000100001

## Next steps

<Admonition type="tip" title="Recommendations">
    - [Specify advanced runtime options.](runtime-options-overview)
    - Practice with primitives by working through the [Cost function lesson](https://learning.quantum.ibm.com/course/variational-algorithm-design/cost-functions#primitives) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile/) section.
    - Try the [Submit pre-transpiled circuits](https://learning.quantum.ibm.com/tutorial/submitting-user-transpiled-circuits-using-primitives) tutorial.
    - Read [Migrate to V2 primitives](/migration-guides/v2-primitives).
    - Understand the [Job limits](/guides/job-limits) when sending a job to an IBM&reg; QPU.
</Admonition>